In [1]:
# imports
import pandas as pd
import os
import numpy as np

In [2]:
# Settings for displaying floats
pd.set_option('display.float_format', '{:,.2f}'.format)

In [3]:
# Datenimport aus allen Dateien, die mit 'waqi-covid-' beginnen
data_folder = './data/'
all_files = [f for f in os.listdir(data_folder) if f.startswith('waqi-covid-')]

# Liste zur Speicherung der DataFrames
dataframes = []

# CSV-Dateien einlesen
for file in all_files:
    file_path = os.path.join(data_folder, file)
    df = pd.read_csv(file_path, comment='#')

    # Alle "wind-gust" Werte in "wind gust" umwandeln
    df["Specie"] = df["Specie"].replace("wind gust", "wind-gust")
    df["Specie"] = df["Specie"].replace("wind speed", "wind-speed")
    
    # Falls die CSV-Datei intern doppelte Zeilen enthält, entfernen
    df = df.drop_duplicates()

    dataframes.append(df)

# Überprüfen, ob Daten vorhanden sind
if not dataframes:
    print("Keine Daten gefunden.")
    df = None
else:
    # Alle DataFrames zu einem großen zusammenfügen
    df = pd.concat(dataframes, ignore_index=True)
    
    # Falls durch das Zusammenfügen doppelte Einträge entstehen, entfernen
    df = df.drop_duplicates()

# Falls Daten importiert wurden, Ausgabe des DataFrames
df


/var/folders/9l/_gqvrbsd14zb0w_nlgtngd_w0000gn/T/ipykernel_62484/74618988.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(dataframes, ignore_index=True)


,Date,Country,City,Specie,count,min,max,median,variance
0,2024-01-03,JP,Fukuoka,no2,386,1.90,16.70,4.70,75.09
1,2024-04-02,JP,Fukuoka,no2,400,1.00,63.10,10.20,722.01
2,2024-11-21,JP,Fukuoka,no2,392,1.00,32.50,8.40,374.60
3,2025-01-05,JP,Fukuoka,no2,396,1.00,21.30,6.50,113.94
4,2025-01-24,JP,Fukuoka,no2,396,1.90,50.10,13.00,"1,155.90"
...,...,...,...,...,...,...,...,...,...
14191264,2018-03-26,IN,Kolkata,no2,24,3.80,5.90,4.60,2.68
14191265,2018-03-03,IN,Kolkata,no2,24,4.80,10.80,5.80,21.75
14191266,2018-05-30,IN,Kolkata,no2,13,0.80,2.50,1.80,2.46
14191267,2018-06-26,IN,Kolkata,no2,21,0.70,5.60,2.30,13.97


In [4]:
# Spalten "variance", "min" und "max" löschen
df = df.drop(columns=['variance', 'min', 'max'], errors="ignore")



In [7]:
# Neu entstandene Duplikate entfernen
df = df.drop_duplicates()
df.shape

(13649037, 6)

In [8]:
# Bestimmen der Städte mit den meisten Messwerten pro Land
city_counts = df.groupby(["Country", "City"]).size().reset_index(name="count")
cities = city_counts.loc[city_counts.groupby("Country")["count"].idxmax()]

In [9]:
# Liste der Städte mit den meisten Messwerten pro Land speichern

# Dateipfad
output_path = './data/city_per_country.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
cities.to_csv(output_path, index=False)
print(f"✅ Datei wurde gespeichert: {output_path}")

✅ Datei wurde gespeichert: ./data/city_per_country.csv


In [10]:
# Filtern nach den Städten mit den meisten Messwerten
city_list = cities['City'].tolist()
df = df[df['City'].isin(city_list)]

In [11]:
df.shape

(2002792, 6)

In [12]:
# Gruppieren nach Datum, Land, Stadt und Messwert; wenn unterschiedliche Werte für Spalte "median" vorhanden, Mittelwert bilden
df = df.groupby(["Date", "Country", "City", "Specie"], as_index=False).agg({"median": "mean"})


In [13]:
# Spalten pivotieren
df = df.pivot(index=["Date", "Country", "City"], columns="Specie", values="median").reset_index()

In [14]:
df

Specie,Date,Country,City,aqi,co,d,dew,humidity,mepaqi,no2,...,pol,precipitation,pressure,psi,so2,temperature,uvi,wd,wind-gust,wind-speed
0,2014-12-29,AT,Vienna,NaN,0.10,NaN,NaN,NaN,NaN,9.00,...,NaN,NaN,NaN,NaN,2.60,NaN,NaN,NaN,NaN,NaN
1,2014-12-29,AU,Brisbane,NaN,1.90,NaN,NaN,NaN,NaN,4.60,...,NaN,NaN,NaN,NaN,1.10,NaN,NaN,NaN,NaN,NaN
2,2014-12-29,BE,Brussels,NaN,0.10,NaN,NaN,NaN,NaN,23.40,...,NaN,NaN,NaN,NaN,1.60,NaN,NaN,NaN,NaN,NaN
3,2014-12-29,BO,Cochabamba,NaN,NaN,NaN,NaN,NaN,NaN,3.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-12-29,BR,São Paulo,NaN,5.50,NaN,NaN,NaN,NaN,17.90,...,NaN,NaN,NaN,NaN,1.10,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225855,2025-02-20,UG,Kampala,NaN,NaN,NaN,20.00,88.00,NaN,NaN,...,NaN,NaN,"1,015.00",NaN,NaN,21.50,NaN,NaN,NaN,2.30
225856,2025-02-20,UZ,Tashkent,NaN,6.80,NaN,2.00,72.50,NaN,4.60,...,NaN,NaN,"1,023.00",NaN,0.60,6.50,NaN,NaN,NaN,1.50
225857,2025-02-20,VN,Hanoi,NaN,5.00,NaN,17.00,96.50,NaN,2.00,...,NaN,NaN,"1,019.00",NaN,6.00,17.00,NaN,NaN,NaN,1.00
225858,2025-02-20,XK,Pristina,NaN,NaN,NaN,NaN,77.10,NaN,NaN,...,NaN,NaN,"1,031.10",NaN,NaN,-9.70,NaN,NaN,0.30,0.10


In [15]:
df.columns

Index(['Date', 'Country', 'City', 'aqi', 'co', 'd', 'dew', 'humidity',
       'mepaqi', 'no2', 'o3', 'pm1', 'pm10', 'pm25', 'pol', 'precipitation',
       'pressure', 'psi', 'so2', 'temperature', 'uvi', 'wd', 'wind-gust',
       'wind-speed'],
      dtype='object', name='Specie')

In [18]:
# Bereinigte Daten speichern

# Dateipfad
output_path = './data/cleaned_data.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df.to_csv(output_path, index=False)
print(f"✅ Datei wurde gespeichert: {output_path}")


✅ Datei wurde gespeichert: ./data/cleaned_data.csv
